## 📦 Dataset Download
<p align="center">
  <a href="https://drive.google.com/drive/folders/1DHVwFYHhsI0_yJycMtlq0ju0qLd5JQsv?usp=sharing" target="_blank">
    <img src="https://img.shields.io/badge/📂_Google_Drive-Dataset-blue?style=for-the-badge&logo=google-drive" alt="Dataset Google Drive">
  </a>
</p>

> ⚠️ **Quan trọng:**  
> Hãy **tải toàn bộ thư mục Dataset về máy** trước khi sử dụng.  
> Không nên chạy hoặc load dữ liệu trực tiếp từ Google Drive.


In [ ]:
# ============================================================
# RNN TEXT CLASSIFICATION - GIẢI THEO TỪNG CÂU TRONG ĐỀ
# RNN (tanh, hidden=2) -> FC(3 lớp) -> softmax -> cross-entropy
# Không dùng thư viện DL; chỉ Python thuần + math.
# Các tham số mà đề KHÔNG cho số (b_h, W_y, b_y) được đặt 0.
# Nếu bạn có số chính thức, thay vào các biến tương ứng là xong.
# ============================================================

import math

# -------------------------
# DỮ LIỆU & EMBEDDING (theo đề)
# -------------------------
tokens = ["Team", "won", "the", "game"]
emb = {
    "Team": [0.1, 0.2],
    "won":  [0.3, 0.4],
    "the":  [0.5, 0.6],
    "game": [0.7, 0.8]
}
# Ground-truth one-hot: [Sports, Politics, Entertainment] = [1,0,0]
y_true = [1, 0, 0]

# -------------------------
# THAM SỐ RNN (theo đề)
# W_xh^T: row1 [0.1, 0.3], row2 [0.2, 0.4]  -> W_xh như dưới
# W_hh^T: row1 [0.5, 0.7], row2 [0.6, 0.8]  -> W_hh như dưới
# -------------------------c
W_xh = [[0.1, 0.2],
        [0.3, 0.4]]  # 2x2
W_hh = [[0.5, 0.6],
        [0.7, 0.8]]  # 2x2
b_h  = [0.1, 0.1]    # 2-dim (đề không ghi -> đặt 0)

h0   = [0.0, 0.0]    # giả định ban đầu h0 = 0 (đề nêu)

# -------------------------
# FULLY CONNECTED (3 lớp)
# Đề chỉ ghi kích thước 3x2, không cho số -> đặt 0
# Nếu bạn có số chính thức, thay vào đây rồi chạy lại.
# -------------------------
W_y = [
    [1.0, 1.1],  # row -> Sports
    [1.2, 1.3],  # row -> Politics
    [1.4, 1.5],  # row -> Entertainment
]  # 3x2
b_y = [0.5, 0.5, 0.5]  # 3-dim

# -------------------------
# HÀM TIỆN ÍCH
# -------------------------
def mat2x2_vec2(M, v):
    return [
        M[0][0]*v[0] + M[0][1]*v[1],
        M[1][0]*v[0] + M[1][1]*v[1],
    ]

def mat_vec(M, v):
    out = []
    for row in M:
        s = 0.0
        for j in range(len(v)):
            s += row[j] * v[j]
        out.append(s)
    return out

def vec_add(a, b):
    return [a[i] + b[i] for i in range(len(a))]

def tanh_vec(v):
    return [math.tanh(v[0]), math.tanh(v[1])]

def softmax(z):
    m = max(z)
    exps = [math.exp(zi - m) for zi in z]
    s = sum(exps)
    return [e/s for e in exps]

def cross_entropy(p, y_onehot):
    idx = y_onehot.index(1)
    return -math.log(p[idx] if p[idx] > 1e-15 else 1e-15)

def pretty(v, nd=6):
    return "[" + ", ".join(f"{x:.{nd}f}" for x in v) + "]"

# ============================================================
# CÂU 1. MÔ TẢ KIẾN TRÚC MÔ HÌNH RNN
# ============================================================
print("========== CÂU 1: MÔ TẢ KIẾN TRÚC ==========")

print("\n1.1) Thành phần & kích thước:")
print("- Input câu: 'Team won the game' -> 4 token, mỗi token -> embedding x_t ∈ R^2")
print("  Embeddings:")
for tk in tokens:
    print(f"    {tk:>5s}: {pretty(emb[tk])}")

print("- RNN (tanh), hidden size = 2:")
print("  h_t = tanh(W_xh x_t + W_hh h_{t-1} + b_h)")
print("  Kích thước: W_xh ∈ R^{2x2}, W_hh ∈ R^{2x2}, b_h ∈ R^{2}, h_t ∈ R^{2}")
print(f"  W_xh = {W_xh}")
print(f"  W_hh = {W_hh}")
print(f"  b_h  = {pretty(b_h)}")
print("  Khởi tạo: h_0 = [0, 0]")

print("- Fully Connected (FC) 3 lớp:")
print("  z = W_y h_T + b_y, với W_y ∈ R^{3x2}, b_y ∈ R^{3}")
print(f"  W_y = {W_y}")
print(f"  b_y = {pretty(b_y)}")
print("- Softmax: p_i = exp(z_i)/sum_j exp(z_j), i ∈ {Sports, Politics, Entertainment}")
print("- Loss: cross-entropy so với one-hot [1,0,0] (Sports)")

print("\n1.2) Dùng h_T để phân loại:")
print("- Lấy h_T (hidden cuối) -> z = W_y h_T + b_y -> p = softmax(z) -> chọn argmax(p)")

# ============================================================
# CÂU 2. MÔ TẢ CÁC THAM SỐ
# ============================================================
print("\n========== CÂU 2: THAM SỐ ==========")

print("2.1) Bảng tham số:")
print("  - W_xh: 2x2 = 4 phần tử (trọng số input->hidden)")
print("  - W_hh: 2x2 = 4 phần tử (trọng số hidden->hidden)")
print("  - b_h : 2   = 2 phần tử (bias RNN)")
print("  - W_y : 3x2 = 6 phần tử (hidden cuối -> 3 lớp)")
print("  - b_y : 3   = 3 phần tử (bias output)")

total_params = 4 + 4 + 2 + 6 + 3
print(f"2.2) Tổng số tham số: 4 + 4 + 2 + 6 + 3 = {total_params}")

# ============================================================
# CÂU 3. TÍNH TOÁN THỦ CÔNG: FORWARD PASS
# ============================================================
print("\n========== CÂU 3: FORWARD PASS ==========")

print("\nBước 1: RNN – Xử lý từng từ")
h_prev = h0[:]
a_list = []
h_list = []

# 3.1: t=1 (Team) — in chi tiết
t = 1; tk = tokens[0]; x_t = emb[tk]
Wx_x = mat2x2_vec2(W_xh, x_t)
Wh_h = mat2x2_vec2(W_hh, h_prev)
a_t  = vec_add(vec_add(Wx_x, Wh_h), b_h)
h_t  = tanh_vec(a_t)
a_list.append(a_t); h_list.append(h_t); h_prev = h_t

print("\n3.1) t=1, token='Team'")
print(f"  x_1                = {pretty(x_t)}")
print(f"  W_xh x_1           = {pretty(Wx_x)}")
print(f"  W_hh h_0           = {pretty(Wh_h)}")
print(f"  a_1 = W_xh x_1 + W_hh h_0 + b_h = {pretty(a_t)}")
print(f"  h_1 = tanh(a_1)                 = {pretty(h_t)}")

# 3.2: t=2 (won) — in chi tiết
t = 2; tk = tokens[1]; x_t = emb[tk]
Wx_x = mat2x2_vec2(W_xh, x_t)
Wh_h = mat2x2_vec2(W_hh, h_prev)
a_t  = vec_add(vec_add(Wx_x, Wh_h), b_h)
h_t  = tanh_vec(a_t)
a_list.append(a_t); h_list.append(h_t); h_prev = h_t

print("\n3.2) t=2, token='won'")
print(f"  x_2                = {pretty(x_t)}")
print(f"  W_xh x_2           = {pretty(Wx_x)}")
print(f"  W_hh h_1           = {pretty(Wh_h)}")
print(f"  a_2 = W_xh x_2 + W_hh h_1 + b_h = {pretty(a_t)}")
print(f"  h_2 = tanh(a_2)                 = {pretty(h_t)}")

# 3.3: t=3,4 — chỉ ghi rõ phép tính (không cần kết quả chi tiết),
# nhưng ta vẫn tính nội bộ để có h_T cho bước 2.
for idx in [2, 3]:
    t = idx + 1
    tk = tokens[idx]
    x_t = emb[tk]
    Wx_x = mat2x2_vec2(W_xh, x_t)
    Wh_h = mat2x2_vec2(W_hh, h_prev)
    a_t  = vec_add(vec_add(Wx_x, Wh_h), b_h)
    h_t  = tanh_vec(a_t)
    a_list.append(a_t); h_list.append(h_t); h_prev = h_t

print("\n3.3) t=3,4 — chỉ cần ghi phép tính:")
print("  Với mỗi t ∈ {3,4}:")
print("    a_t = W_xh x_t + W_hh h_{t-1} + b_h")
print("    h_t = tanh(a_t)")
print("  (Đã tính nội bộ để dùng cho bước 2, không in chi tiết như yêu cầu)")

h_T = h_prev
print("\n=> Hidden state cuối cùng h_T =", pretty(h_T))

# Bước 2: Phân loại chủ đề
print("\nBước 2: Phân loại chủ đề")
z = mat_vec(W_y, h_T)
z = vec_add(z, b_y)
p = softmax(z)

print("\n3.4) Logits & Softmax")
print("  z = W_y h_T + b_y =", pretty(z))
print("  p = softmax(z)    =", pretty(p), f"(tổng={sum(p):.6f})")

L = cross_entropy(p, y_true)
print("\n3.5) Cross-entropy loss")
print("  Ground truth [Sports, Politics, Entertainment] =", y_true)
print("  Loss = -log(p[Sports]) =", f"{L:.6f}")

# ============================================================
# CÂU 4. GIẢI THÍCH CƠ CHẾ HOẠT ĐỘNG
# ============================================================
print("\n========== CÂU 4: GIẢI THÍCH CƠ CHẾ ==========")

print("\n4.1) RNN làm gì & ý nghĩa h_T?")
print("  RNN đọc tuần tự các từ, cập nhật h_t từ x_t và h_{t-1}.")
print("  h_T (hidden cuối) tóm tắt toàn câu, chứa thông tin chủ đề để phân loại.")

print("\n4.2) Vì sao có vanishing gradient & ảnh hưởng?")
print("  Đạo hàm qua nhiều bước nhân với ma trận và đạo hàm tanh (<1) làm gradient nhỏ dần.")
print("  Kết quả: khó học phụ thuộc dài hạn, có thể bỏ lỡ từ khóa xa quyết định chủ đề.")

print("\n4.3) Cross-entropy đo lường gì?")
print("  Đo độ sai khác giữa phân phối dự đoán và nhãn thật one-hot;")
print("  loss thấp khi xác suất cho nhãn đúng cao.")

print("\n4.4) So sánh nhanh RNN vs LSTM:")
print("  Giống: xử lý chuỗi từng bước, dùng hidden state, áp dụng cho phân loại văn bản.")
print("  Khác: LSTM có cell state và các cổng (input/forget/output) giúp giữ thông tin dài hạn,")
print("        giảm vanishing gradient; RNN thuần đơn giản hơn nhưng khó học phụ thuộc xa.")

========== CÂU 1: MÔ TẢ KIẾN TRÚC ==========

1.1) Thành phần & kích thước:
- Input câu: 'Team won the game' -> 4 token, mỗi token -> embedding x_t ∈ R^2
  Embeddings:
     Team: [0.100000, 0.200000]
      won: [0.300000, 0.400000]
      the: [0.500000, 0.600000]
     game: [0.700000, 0.800000]
- RNN (tanh), hidden size = 2:
  h_t = tanh(W_xh x_t + W_hh h_{t-1} + b_h)
  Kích thước: W_xh ∈ R^{2x2}, W_hh ∈ R^{2x2}, b_h ∈ R^{2}, h_t ∈ R^{2}
  W_xh = [[0.1, 0.2], [0.3, 0.4]]
  W_hh = [[0.5, 0.6], [0.7, 0.8]]
  b_h  = [0.100000, 0.100000]
  Khởi tạo: h_0 = [0, 0]
- Fully Connected (FC) 3 lớp:
  z = W_y h_T + b_y, với W_y ∈ R^{3x2}, b_y ∈ R^{3}
  W_y = [[1.0, 1.1], [1.2, 1.3], [1.4, 1.5]]
  b_y = [0.500000, 0.500000, 0.500000]
- Softmax: p_i = exp(z_i)/sum_j exp(z_j), i ∈ {Sports, Politics, Entertainment}
- Loss: cross-entropy so với one-hot [1,0,0] (Sports)

1.2) Dùng h_T để phân loại:
- Lấy h_T (hidden cuối) -> z = W_y h_T + b_y -> p = softmax(z) -> chọn argmax(p)

========== CÂU 2: THAM S